<a href="https://colab.research.google.com/github/aldrich1221/GNNforStockPrediction/blob/main/GNN_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install quandl
!pip install torch
!pip install --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
#!pip install --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install torch-geometric
!pip install networkx
!pip install matplotlib
!pip install numpy
!pip install sklearn
!pip install arch
!pip install tushare
!pip install pyflux
!pip install sklearn
!pip install apyori

Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html


Parameter Setting

In [ ]:
ClassNum=5
ClassThreshold=[-0.01,-0.0025,0.0025,0.01]  #lenth must be ClassNum-1
AssociationPara=[0.1,0.1,1.7,0.01]
Corr=0.75
fakeNode=True

Packages

In [ ]:
import pandas as pd
import numpy as np
import quandl, datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
import torch
from torch_scatter import scatter_add
from torch_geometric.nn import MessagePassing
import math

StoragePath="/content/drive/MyDrive/MyCoLabStorage/GraphNeuralNetworkData"

API_Key="NJB8FEA7zxkhzwHsj5Z7"

In [ ]:
def assignClass(AllLableFrame,classNum,ClassThreshold):
   import numpy as np
    
   for i in range(len(AllLableFrame)):
      for j in range(len(AllLableFrame.columns)):
        value=AllLableFrame.loc[i][j]
        newValue=0
        for iclass in range(classNum):
          if iclass==0:

            if value<=ClassThreshold[iclass]:
              newValue=iclass
          elif iclass==classNum-1:
            if value>ClassThreshold[iclass-1]:
              newValue=iclass
          else:
            if (value>ClassThreshold[iclass-1])&(value<=ClassThreshold[iclass]):
              newValue=iclass   
        AllLableFrame.loc[i][j]=newValue
    
   return AllLableFrame
def columnsRename(data):
  if 'Bid Average' in data.columns:
    data.rename(columns={'Bid Average': 'Value'}, inplace=True)
  if 'Close' in data.columns:
    data.rename(columns={'Close': 'Value'}, inplace=True)
  if 'Last' in data.columns:
    data.rename(columns={'Last': 'Value'}, inplace=True)
  if '5 YR' in data.columns:
    data.rename(columns={'5 YR': 'Value'}, inplace=True)
  if 'Short Volume' in data.columns:
    data.rename(columns={'Short Volume': 'Value'}, inplace=True)
  return data  

APISymbolListFromYahoo=['OXY','CVX','XOM','COP','SLB','EOG','PSX','MPC','KMI','WMB','PXD','VER','ORA','FSLR','CEN','BLX']
SymbolListFromYahoo=['OXY','CVX','XOM','COP','SLB','EOG','PSX','MPC','KMI','WMB','PXD','VER','ORA','FSLR','CEN','BLX']

APISymbolListFromQuandl=['PERTH/GOLD_USD_D','PERTH/SLVR_USD_D','CHRIS/CME_CL1','CHRIS/CME_ES1','EURONEXT/RDSA','CHRIS/CME_S1','CHRIS/CME_C1','CHRIS/CME_W1']
SymbolListFromQuandl=['GLD','Silver','Oil','S&P500','RDSA','SOYB','CORN','WHEAT']

SymbolList=SymbolListFromYahoo+SymbolListFromQuandl
APISymbolList=APISymbolListFromYahoo+APISymbolListFromQuandl


AllDataFrame=pd.DataFrame()
NoMADataFrame=pd.DataFrame()
d=pd.DataFrame()
columns=[]
first=[]
Nornalizerlist=[]
for symbol in SymbolList:
  dataFrame=pd.read_csv(StoragePath+"/allStockRawData/"+symbol+".csv")
  
  dataFrame=columnsRename(dataFrame)
  columnPrice=symbol+"_Value"
  # columnVolumn=symbol+"_volume"
  columnFuturePrice=symbol+"_ValueFuture"
  columns.append(columnFuturePrice)
  check_for_nan_price = dataFrame['Value'].isnull()
  #check_for_nan_volume = dataFrame['Adj. Close'].isnull()

  dataFrame['Value'][check_for_nan_price]=np.median(dataFrame['Value'])
  #dataFrame['Adj. Volume'][check_for_nan_volume]=np.median(dataFrame['Adj. Volume'])
  A=dataFrame['Value'][~check_for_nan_price]
 
  transformer = preprocessing.Normalizer().fit([A[:200]])

  #Not do normalize
  dataFrame['Value'][~check_for_nan_price]=transformer.transform([dataFrame['Value'][~check_for_nan_price]])[0]
  
  
  Nornalizerlist.append(transformer)

  NoMADataFrame[columnPrice]=dataFrame['Value']
  AllDataFrame[columnPrice]=dataFrame['Value']
  AllDataFrame[symbol+"_MA5"]=AllDataFrame[columnPrice].rolling(window=5).mean()


  #AllDataFrame[columnVolumn]=dataFrame['Adj. Volume']
  first.append(dataFrame['Value'][0])
  
  
  d[columnFuturePrice]=dataFrame['Value'][1:]
  

LabelFrame=pd.DataFrame([first],columns=columns)

LabelFrame=pd.concat([LabelFrame, d])

AllLableFrame=(LabelFrame.diff()/LabelFrame)[6:]
AllDailyReturnFrame = (AllDataFrame.diff() / AllDataFrame)[5:]
AllLableFrame=AllLableFrame.reset_index(drop=True)
AllDailyReturnFrame=AllDailyReturnFrame.reset_index(drop=True)
NoMADataFrame=NoMADataFrame.diff()/NoMADataFrame



AllYFrame=AllLableFrame
AllLableFrame=assignClass(AllLableFrame,ClassNum,ClassThreshold)

print(AllLableFrame.head(4))
print(AllDailyReturnFrame.head(5))
print(len(AllDailyReturnFrame))
# A=[]
# Class=[]
# for i in range(ClassNum):
#   Class.append([])
# for i in AllLableFrame.columns:
#   count=AllLableFrame[i].value_counts()
#   A.append(AllLableFrame[i].value_counts())
#   for j in range(ClassNum):

#     print(Class[j],"count[j]",count[j])
#     Class[j].append(count[j])
    
# Total=np.sum(np.sum(Class))
# print(np.sum(Class[0])/Total,np.sum(Class[1])/Total,np.sum(Class[2])/Total,np.sum(Class[3])/Total,np.sum(Class[4])/Total)
# print(A)

In [ ]:

def AddGrachFeatures(Add,AllDailyReturnFrame):
  if Add==False:
    return AllDailyReturnFrame
  
  from scipy import  stats
  import statsmodels.api as sm 
  import numpy as np
  import pandas as pd
  import matplotlib.pyplot as plt
  import arch  
  

  fig = plt.figure(figsize=(20,5))
  for isymbol in range(len(SymbolList)):
    symbol=SymbolList[isymbol]
   

    
    data=AllDailyReturnFrame[symbol+"_Value"][:100]
    print(data)
    import numpy as np
    import pyflux as pf
    import pandas as pd
    
    from datetime import datetime
    import matplotlib.pyplot as plt
    %matplotlib inline
    model = pf.GARCH(np.array(data),p=1,q=1)
    x = model.fit()
    x.summary()
    newFeatures=model.predict(h=len(AllDailyReturnFrame)-100).values
    
    Temp=np.zeros(100).tolist()
    for eachfeature in newFeatures:
      Temp.append(eachfeature[0])
    AllDailyReturnFrame[symbol+"_Garch"]=Temp
  AllDailyReturnFrame.to_csv(StoragePath+"AllDailyReturnFrame.csv")
    
  return AllDailyReturnFrame

  
def AddRandomFeature(Add,AllDailyReturnFrame):
  if Add==False:
    return AllDailyReturnFrame
  for isymbol in range(len(SymbolList)):
    symbol=SymbolList[isymbol]
   
    
    
    data=AllDailyReturnFrame[symbol+"_Value"]
    NewFeature=[data[0] ]
    for ieachdata in range(1,len(data)):
      epsilon=np.random.rand(1)[0]*np.abs(data[ieachdata]-data[ieachdata-1])
      NewFeature.append(np.random.randn(1)[0]*epsilon/2+data[ieachdata])
    
    AllDailyReturnFrame[symbol+"_NewFeature"]=NewFeature
  return AllDailyReturnFrame
#AllDailyReturnFrame=AddGrachFeatures(False,AllDailyReturnFrame)
AllDailyReturnFrame=AddRandomFeature(True,AllDailyReturnFrame)
print(AllDailyReturnFrame.head())


In [ ]:
def ExploreMissingValue(trainData_0):
    missing=pd.concat([trainData_0.isnull().any(),trainData_0.count()],axis=1)
    column=['miss or not','count']
    
    missing=pd.DataFrame(list(missing.values),index=list(missing.index),columns=column)
    max=missing['count'].max()
    missing['missing count']=max-missing['count']
    missing['missing ratio']=missing['missing count']/max
    miss=missing[missing['count'] < max]
    print(miss)
    
    return np.array(missing.index[missing['miss or not']==True])  ##return the index with missing value
def FillNullWithMedian(DataFrame):
  for col in DataFrame.columns:
    NullIndex=np.where(np.array(DataFrame[col].isnull()))[0]
    
    Median=np.nanmedian(DataFrame[col])
    
    DataFrame[col][NullIndex]=Median

FillNullWithMedian(AllDailyReturnFrame)   
ExploreMissingValue(AllDailyReturnFrame)
FillNullWithMedian(NoMADataFrame)
fig , ax = plt.subplots()
fig.subplots_adjust(hspace=0.1, wspace=0.1) 
ax2 = ax.twinx()


for icol in range(int(len(AllDailyReturnFrame.columns)/2)):
   #plt.subplot(len(AllDailyReturnFrame.columns)/2,len(AllDailyReturnFrame.columns)/2,icol+1)
   plt.plot(AllDailyReturnFrame[AllDailyReturnFrame.columns[icol]])
plt.show()

AllDailyReturnFrame.describe()

In [ ]:

def AssociationAnalysis(AllDailyReturnFrame_AssociationAnalysis,StartEdgeCorr,EndEdgeCorr,GoAssociationAnalysis):
  if GoAssociationAnalysis==False:
    return [],[]
  from apyori import apriori
  import pandas as pd
  # from mlxtend.frequent_patterns import apriori
  # from mlxtend.frequent_patterns import association_rules
  AllData=[]
  for date in AllDailyReturnFrame_AssociationAnalysis.index:
    DateData=AllDailyReturnFrame_AssociationAnalysis.loc[date]
    UpData=[]
    NoChangeData=[]
    DownData=[]
    for eachSymbol in DateData.index:
      if DateData[eachSymbol]>AssociationPara[3]:
        
        UpData.append(eachSymbol.split('_')[0])
        
      elif DateData[eachSymbol]<AssociationPara[3]:
        DownData.append(eachSymbol.split('_')[0])
      else:
        NoChangeData.append(eachSymbol.split('_')[0])
    
    
    AllData.append(UpData)
    #AllData.append(NoChangeData)
    AllData.append(DownData)
    
  
  data = AllData


  
  association_rules = apriori(data, min_support=AssociationPara[0], min_confidence=AssociationPara[1], min_lift=AssociationPara[2], max_length=2) 
  association_results = list(association_rules)

 
  for eachResult in association_results:
    resultitems=list(eachResult.items)
    item1Index=np.where(np.array(SymbolList)==resultitems[0])[0]
    item2Index=np.where(np.array(SymbolList)==resultitems[1])[0]
    print("Add",item1Index,item2Index)
    if len(item1Index)>0:
      if len(item2Index)>0:
        StartEdgeCorr.append(item1Index[0])
        EndEdgeCorr.append(item2Index[0])
    if len(item1Index)>0:
      if len(item2Index)>0:
        StartEdgeCorr.append(item2Index[0])
        EndEdgeCorr.append(item1Index[0])
    #print(product) # ex. RelationRecord(items=frozenset({'Basketball', 'Socks'}), support=0.25, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Basketball'}), items_add=frozenset({'Socks'}), confidence=0.5, lift=2.0), OrderedStatistic(items_base=frozenset({'Socks'}), items_add=frozenset({'Basketball'}), confidence=1.0, lift=2.0)])
    product=eachResult
    pair = eachResult[0] 
    ##print(pair) ## ex. frozenset({'Basketball', 'Socks'})
    products = [x for x in pair]
    print(products) # ex. ['Basketball', 'Socks']
    print("Rule: " + products[0] + " →→   " + products[1])
    print("Support: " + str(product[1]))
    print("Lift: " + str(product[2][0][3]))
   

  print(len(EndEdgeCorr))
 
  return StartEdgeCorr,EndEdgeCorr


CorrDataFrame=NoMADataFrame.loc[:200].corr(method ='pearson')
print(CorrDataFrame)
# print(np.where(np.array(CorrDataFrame)>0.7)[0])

StartEdgeCorr=np.where(np.array(CorrDataFrame)>Corr )[0]
EndEdgeCorr=np.where(np.array(CorrDataFrame)>Corr)[1]
print(StartEdgeCorr)
StartEdgeFromApriori=[]
EndEdgeFromApriori=[]

for i in range(len(StartEdgeCorr)):
  if StartEdgeCorr[i]!=EndEdgeCorr[i]:
    StartEdgeFromApriori.append(StartEdgeCorr[i])
    EndEdgeFromApriori.append(EndEdgeCorr[i])

print("Cor:",StartEdgeFromApriori)
print("cor:",EndEdgeFromApriori)



StartEdgeFromApriori,EndEdgeFromApriori=AssociationAnalysis(NoMADataFrame.loc[:100],StartEdgeFromApriori,EndEdgeFromApriori,True)


# StartEdgeFromApriori=[]
# EndEdgeFromApriori=[]
print(StartEdgeFromApriori)
print(EndEdgeFromApriori)

Model definition

In [ ]:
class MyGNN(MessagePassing):
    def __init__(self, in_channels, out_channels,havabias):
        super(MyGNN, self).__init__(aggr='add') 
        self.weight1=torch.nn.Parameter(torch.Tensor(in_channels, 64))
        self.bias1 = torch.nn.Parameter(torch.Tensor(64))
        self.weight2 = torch.nn.Parameter(torch.Tensor(64, out_channels))
        self.bias2= torch.nn.Parameter(torch.Tensor(out_channels))
        self.edge_weight = torch.ones((edge_index.size(1),),
                                 dtype=x.dtype,
                                 device=edge_index.device)  
        if havabias:
            self.bias = torch.nn.Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)
            

        if self.weight1 is not None:
          stdv = math.sqrt(3.0 / (self.weight1.size(-2) + self.weight1.size(-1)))
          self.weight1.data.normal_(-stdv, stdv)
        if self.weight2 is not None:
          stdv = math.sqrt(3.0 / (self.weight2.size(-2) + self.weight2.size(-1)))
          self.weight2.data.normal_(-stdv, stdv)
        
        if self.bias1 is not None:
          self.bias1.data.fill_(0)
        if self.bias2 is not None:
          self.bias2.data.fill_(0)
        if self.bias is not None:
          self.bias1.data.fill_(0)
          
    def forward(self, x, edge_index):
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        x = torch.matmul(x, self.weight1)+self.bias1  ##fully connected layer1
        x=F.relu(x)
        x = torch.matmul(x, self.weight2)+self.bias2  ##fully connected layer2
       

        row, col = edge_index
        deg = degree(row, x.size(0), dtype=x.dtype)
        
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x,
                              norm=norm)
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j
   
class Net(torch.nn.Module):
    def __init__(self, dataset):
        
        super(Net, self).__init__()
        self.conv1 = MyGNN(dataset.num_node_features, 256,havabias=False)
        self.conv2 = MyGNN(256, dataset.num_classes,havabias=True)
        self.leakyrelu=torch.nn.LeakyReLU(0.01)
        #self.conv1 = GATConv(dataset.num_node_features, 16)
        #self.conv2 = GATConv(16, dataset.num_classes)
        #self.lstm = nn.LSTM(256, 256, dropout=0.2)
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x= self.leakyrelu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [ ]:

from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing,TopKPooling
from torch_geometric.utils import add_self_loops, degree
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.utils import remove_self_loops, add_self_loops
from torch_geometric.data import Data
from torch_geometric.nn import FastRGCNConv,GMMConv,GCN2Conv,GATConv
import torch.nn as nn
   

def generateDataMethod2(X,Y,idx,NDays,RelationMatrix,ClassNum):
    def Convert2GraphData(GraphNodeX,GraphNodeY,GraphStartEdge,GraphEndEdge,trainMask,testMask,ClassNum,GraphNodeText):
      
      x = torch.tensor(GraphNodeX, dtype=torch.float)
      
      y = torch.tensor(GraphNodeY, dtype=torch.long)
     
      edge_index = torch.tensor([GraphStartEdge,
                            GraphEndEdge], dtype=torch.long)
      
      data = Data(x=x ,edge_index=edge_index, y=y,test_mask=testMask, train_mask=trainMask,num_classes=ClassNum,node_text=GraphNodeText)
      return data
    def Assign(value):
          newValue=0
          if value<=-0.03:
            newValue=0
          elif (value>-0.03)&(value<=-0.01):
            newValue=1
          elif (value>-0.01)&(value<=0.01):
            newValue=2
          elif (value>0.01)&(value<=0.03):
            newValue=3
          else:
            newValue=4
          return newValue
    N=len(X)-1
    
    TrainIndexFrom=100+idx
    TrainIndexTo=N-(NDays-idx)-1
    TestIndexFrom=100+1+idx
    TestIndexTo=N-(NDays-idx)


    trainX=X.loc[np.array(range(TrainIndexFrom,TrainIndexTo)).tolist()].reset_index ()
    testX=X.loc[np.array(range(TestIndexFrom,TestIndexTo)).tolist()].reset_index ()
    
    GraphNodeX_train=[]
    GraphNodeX_test=[]
    GraphNodeText=[]

    GraphNodeY_train=np.array(Y.loc[TrainIndexTo].values).tolist()
    GraphNodeY_test=np.array(Y.loc[TestIndexTo].values).tolist()
    
    GraphStartEdge=[]
    GraphEndEdge=[]
    
    for isymbol in range(len(SymbolList)):
      symbol=SymbolList[isymbol]
      columnPrice=symbol+"_Value"
      columnFeature1=symbol+"_NewFeature"
      columnVolumn=symbol+"_volume"
      columnFuturePrice=symbol+"_ValueFuture"
      #combineTrainFeatures=trainX[columnPrice].tolist()+trainX[columnVolumn].tolist()
      #combineTestFeatures=testX[columnPrice].tolist()+testX[columnVolumn].tolist()

      combineTrainFeatures=trainX[columnPrice].tolist()
      combineTestFeatures=testX[columnPrice].tolist()

      GraphNodeX_train.append(combineTrainFeatures)
      GraphNodeX_test.append(combineTestFeatures)
      GraphNodeText.append(symbol)
      
      #GraphStartEdge,GraphEndEdge=generateEdge(GraphStartEdge,GraphEndEdge,isymbol,symbol)
   
    GraphStartEdge=StartEdgeFromApriori
    GraphEndEdge=EndEdgeFromApriori
    
    if fakeNode==True:
      for isymbol in range(len(SymbolList)):
        symbol=SymbolList[isymbol]
        columnPrice=symbol+"_Value"
        columnFeature1=symbol+"_MA5"
        columnVolumn=symbol+"_volume"
        columnFuturePrice=symbol+"_ValueFuture"
        #combineTrainFeatures=trainX[columnPrice].tolist()+trainX[columnVolumn].tolist()
        #combineTestFeatures=testX[columnPrice].tolist()+testX[columnVolumn].tolist()

        combineTrainFeatures=trainX[columnFeature1].tolist()
        combineTestFeatures=testX[columnFeature1].tolist()

        GraphNodeX_train.append(combineTrainFeatures)
        GraphNodeX_test.append(combineTestFeatures)
        GraphNodeText.append(symbol+"_Fake")


        GraphStartEdge.append(isymbol)
        GraphEndEdge.append(isymbol+len(SymbolList))
        GraphStartEdge.append(isymbol+len(SymbolList))
        GraphEndEdge.append(isymbol)


        GraphNodeY_train.append(Assign(trainX[columnFeature1].tolist()[-1]))
        GraphNodeY_test.append(Assign(testX[columnFeature1].tolist()[-1]))

    trainData=Convert2GraphData(GraphNodeX_train,GraphNodeY_train,GraphStartEdge,GraphEndEdge,np.array(range(len(GraphNodeY_train))).tolist(),[],ClassNum,GraphNodeText)
    testData =Convert2GraphData(GraphNodeX_test,GraphNodeY_test,GraphStartEdge,GraphEndEdge,[],np.array(range(len(GraphNodeY_test))).tolist(),ClassNum,GraphNodeText)
    
    return trainData,testData
def plot_dataset(dataset):
    import matplotlib.pyplot as plt
    import networkx as nx
    edges_raw = dataset.edge_index.numpy()
    edges = [(x, y) for x, y in zip(edges_raw[0, :], edges_raw[1, :])]
    labels = dataset.y.numpy()
    labelsDict={}
    for i in range(len(dataset.node_text)):
      labelsDict[i]=dataset.node_text[i]
    G = nx.Graph()
   
    G.add_nodes_from(list(range(np.max(edges_raw))))
    G.add_edges_from(edges)
    plt.subplot(111)
    options = {
                'node_size': 50,
                'width': 0.6,
    }
    with_labels=True
    if len(dataset.node_text)>20:
      with_labels=False
    values = labels

    # actual code
    colorLeft = np.array([112, 224, 112])
    colorRight = np.array([224, 112, 112])
    scaled = MinMaxScaler().fit_transform(values.reshape(-1, 1))
    colors = np.array([a * colorRight + (1 - a) * colorLeft for a in scaled], dtype = np.int64)
    nx.draw(G, with_labels=with_labels,labels=labelsDict, node_color=colors/255, cmap=plt.cm.tab10, font_weight='bold', **options)
    
    # fig.canvas.mpl_connect("motion_notify_event", hover)

    plt.show()
    

def test(data, train=True):
    model.eval()

    correct = 0
    pred = model(data).max(dim=1)[1]
   

    if train:
        correct += pred[:len(SymbolList)].eq(data.y[:len(SymbolList)]).sum().item()
        return correct / (len(data.y[:len(SymbolList)]))
    else:
        
        #print("pred: ",pred[data.test_mask],"True: ",data.y[data.test_mask])
        correct += pred[:len(SymbolList)].eq(data.y[:len(SymbolList)]).sum().item()
        return correct / (len(data.y[:len(SymbolList)]))

def targetAssetTest(data,targetIndex):
  model.eval()

  correct = 0
  pred = model(data).max(dim=1)[1]

  return pred[data.test_mask][targetIndex],data.y[data.test_mask][targetIndex]


def train(data1,data2, plot=False):
    train_accuracies, test_accuracies = list(), list()
    for epoch in range(200):
            
            model.train()
            optimizer.zero_grad()
            out = model(data1)
            loss = F.nll_loss(out[data1.train_mask], data1.y[data1.train_mask])
            loss.backward()
            optimizer.step()

            train_acc = test(data1)
            test_acc = test(data2, train=False)

            train_accuracies.append(train_acc)
            test_accuracies.append(test_acc)
            print('Epoch: {:03d}, Loss: {:.5f}, Train Acc: {:.5f}, Test Acc: {:.5f}'.
                  format(epoch, loss, train_acc, test_acc))

    if plot:
        plt.plot(train_accuracies, label="Train accuracy")
        plt.plot(test_accuracies, label="Validation accuracy")
        plt.xlabel("# Epoch")
        plt.ylabel("Accuracy")
        plt.legend(loc='upper right')
        plt.show()
    return train_accuracies,test_accuracies

def generateEdge(GraphStartEdge,GraphEndEdge,ThisNodeIndex,symbol):
    #Node that ,to draw graph,the edge must be a bidirectional link
    print("generateEdge",GraphStartEdge,GraphEndEdge,ThisNodeIndex,symbol)
    if symbol=='GLD':   
      GraphStartEdge.append(ThisNodeIndex+1) #GLD->Silver
      
      GraphEndEdge.append(ThisNodeIndex)
      
      GraphStartEdge.append(ThisNodeIndex) #Silver->GLD
      
      GraphEndEdge.append(ThisNodeIndex+1)
      

      GraphStartEdge.append(ThisNodeIndex+2) #GLD->Oil
      GraphEndEdge.append(ThisNodeIndex)
      GraphStartEdge.append(ThisNodeIndex) #Oil->GLD
      GraphEndEdge.append(ThisNodeIndex+2)

    elif symbol=='Silver':
      GraphStartEdge.append(ThisNodeIndex+1) #Silver->Oil
      GraphEndEdge.append(ThisNodeIndex)
      GraphStartEdge.append(ThisNodeIndex) #Oil->Silver
      GraphEndEdge.append(ThisNodeIndex+1)
    elif symbol=='Oil':
      GraphStartEdge.append(ThisNodeIndex) #Oil->RDSA
      GraphEndEdge.append(ThisNodeIndex+3)
      GraphStartEdge.append(ThisNodeIndex+3) #RDSA->Oil
      GraphEndEdge.append(ThisNodeIndex)
    elif symbol=='VIX':
      GraphStartEdge.append(ThisNodeIndex) #VIX->DOWJOHN
      
      GraphEndEdge.append(ThisNodeIndex+1)
      
      GraphStartEdge.append(ThisNodeIndex+1) #DOWJOHN->VIX
      
      GraphEndEdge.append(ThisNodeIndex)
      


    return GraphStartEdge,GraphEndEdge
   

#generateDataMethod1
def generateDataMethod1(X,Y,idx,NDays,RelationMatrix,ClassNum):
    
    def Convert2GraphData(GraphNodeX,GraphNodeY,GraphStartEdge,GraphEndEdge,trainMask,testMask,ClassNum,GraphNodeText):
      x = torch.tensor(GraphNodeX, dtype=torch.float)
      y = torch.tensor(GraphNodeY, dtype=torch.long)

      edge_index = torch.tensor([GraphStartEdge,
                            GraphEndEdge], dtype=torch.long)

      data = Data(x=x ,edge_index=edge_index, y=y,test_mask=testMask, train_mask=trainMask,num_classes=ClassNum,node_text=GraphNodeText)
      return data
    
    edgeStart=RelationMatrix[0][0]
    edgeEnd=RelationMatrix[0][1]

    N=len(X)-1
    # trainX=X[idx:N-(NDays-idx)]
    # trainY=Y.loc[idx:N-(NDays-idx)]
    # testX=X[1+idx:N-(NDays-idx)+1]
    # testY=Y.loc[N-(NDays-idx)+1]
    
    GraphNodeX=[]
    GraphNodeY=[]
    GraphStartEdge=[]
    GraphEndEdge=[]
    GraphNodeText=[]
    
    Test_Mask=[]
    Discard_Mask=[]
    Train_Mask=[]
    for time in range(1,len(X)):
    #for time in range(1,3):
      if time>N-(NDays-idx) :     #The data we want to predict
        ##Start Test Point
        mask=True
      else:
        mask=False
      if time>N-(NDays-idx)+1:   #The data comes from future
        dis_mask=True
      else:
        dis_mask=False
      
      if time>=idx:
        train_mask=True
      else:
        train_mask=False
      
      eachTimeStartIndex=len(GraphNodeY)
      ## eachTimeStartIndex-len(SymbolList):GLD_t-1,eachTimeStartIndex+1-len(SymbolList):Silver_t-1 ....
      #SymbolList=['GLD','Silver','Oil','VIX','DOWJOHN','RDSA']
     
      for isymbol in range(len(SymbolList)):
        symbol=SymbolList[isymbol]
        Test_Mask.append(mask)
        Discard_Mask.append(dis_mask)
        Train_Mask.append(train_mask)
        ##Features in each Node
        #nodeFeatures=[X.loc[time][symbol+"_Value"],X.loc[time-1][symbol+"_Value"],X.loc[time][symbol+"_MA5"],X.loc[time-1][symbol+"_MA5"]]
        nodeFeatures=X.loc[time].tolist()+X.loc[time-1].tolist()
        GraphNodeText.append(symbol+"_time"+str(time))
        GraphNodeX.append(nodeFeatures)
        GraphNodeY.append(Y.loc[time][symbol+"_ValueFuture"])
        ThisNodeIndex=eachTimeStartIndex+isymbol
        
        GraphStartEdge,GraphEndEdge=generateEdge(GraphStartEdge,GraphEndEdge,ThisNodeIndex,symbol)
       
          

    
    testMask=list(set(np.where(np.array(Test_Mask)==True)[0] .tolist()).intersection(set(np.where(np.array(Discard_Mask)==False)[0].tolist())))
    trainMask=list(set(np.where(np.array(Train_Mask)==True)[0] .tolist()).intersection(set(np.where(np.array(Test_Mask)==False)[0].tolist())))

   
   
    return Convert2GraphData(GraphNodeX,GraphNodeY,GraphStartEdge,GraphEndEdge,trainMask,testMask,ClassNum,GraphNodeText)


  
def OverallGraph(Overall):
    plt.hist(Overall, label="test accuracy", bins = ClassNum)
    
    #plt.bar(Overall, label="test accuracy")
   
    plt.xlabel("accuracy range")
    plt.ylabel("Day Count")
    plt.legend(loc='upper right')
    plt.show()
def BuyHoldTrading(MethodDict,price):
  AssetAmout=MethodDict['AssetStockAmout'][-1]
  AssetValue=MethodDict['AssetValue'][-1]
  MethodDict['AssetValue'].append(AssetValue)
  MethodDict['AssetStockAmout'].append(AssetAmout)
  MethodDict['Action'].append(0)
  MethodDict['TotalAssetValue'].append(MethodDict['AssetStockAmout'][-1]*price+MethodDict['AssetValue'][-1])
  return MethodDict
def Trading(data,target,price,MethodDict):

  model.eval()

  correct = 0
  pred = model(data).max(dim=1)[1]

  action=pred[data.test_mask][target]
  if action>np.floor(ClassNum/2):
    AssetAmout=MethodDict['AssetStockAmout'][-1]+1

    AssetValue=MethodDict['AssetValue'][-1]-price


    MethodDict['AssetValue'].append(AssetValue)
    MethodDict['AssetStockAmout'].append(AssetAmout)
    MethodDict['Action'].append(action)
    MethodDict['TotalAssetValue'].append(MethodDict['AssetStockAmout'][-1]*price+MethodDict['AssetValue'][-1])

    return MethodDict
  elif action<np.floor(ClassNum/2):
    AssetAmout=MethodDict['AssetStockAmout'][-1]-1

    AssetValue=MethodDict['AssetValue'][-1]+price


    MethodDict['AssetValue'].append(AssetValue)
    MethodDict['AssetStockAmout'].append(AssetAmout)
    MethodDict['Action'].append(action)
    MethodDict['TotalAssetValue'].append(MethodDict['AssetStockAmout'][-1]*price+MethodDict['AssetValue'][-1])
    return MethodDict
  else:
    AssetAmout=MethodDict['AssetStockAmout'][-1]

    AssetValue=MethodDict['AssetValue'][-1]


    MethodDict['AssetValue'].append(AssetValue)
    MethodDict['AssetStockAmout'].append(AssetAmout)
    MethodDict['Action'].append(action)
    MethodDict['TotalAssetValue'].append(MethodDict['AssetStockAmout'][-1]*price+MethodDict['AssetValue'][-1])
    return MethodDict
def UpdatePriceList(AllDailyReturnFrame,stockIndex,testDayIndex,Method,StockPriceList):
  columnsName=AllDailyReturnFrame.columns[stockIndex*2]
  if Method==1:
    TestDayReturn=AllDailyReturnFrame.loc[testDayIndex][columnsName]
  elif Method==2:
    TestDayReturn=AllDailyReturnFrame.loc[testDayIndex-1][columnsName]
  StockPrice=StockPriceList[-1]*(1+TestDayReturn)
  StockPriceList.append(StockPrice)
  return StockPriceList
def ClearPosition(MethodDict,StockPriceList):

  if MethodDict['AssetStockAmout'][-1]>0:
    MethodDict['AssetValue'].append(MethodDict['AssetValue'][-1]+StockPriceList[-1]*MethodDict['AssetStockAmout'][-1])
    MethodDict['AssetStockAmout'].append(0)
    MethodDict['Action'].append(-1)
    MethodDict['TotalAssetValue'].append(MethodDict['AssetValue'][-1]+StockPriceList[-1]*MethodDict['AssetStockAmout'][-1])
  elif MethodDict['AssetStockAmout'][-1]<0:
    MethodDict['AssetValue'].append(MethodDict['AssetValue'][-1]-StockPriceList[-1]*MethodDict['AssetStockAmout'][-1])
    MethodDict['AssetStockAmout'].append(0)
    MethodDict['Action'].append(1)
    MethodDict['TotalAssetValue'].append(MethodDict['AssetValue'][-1]-StockPriceList[-1]*MethodDict['AssetStockAmout'][-1])
  else:
    MethodDict['AssetValue'].append(MethodDict['AssetValue'][-1])
    MethodDict['AssetStockAmout'].append(0)
    MethodDict['Action'].append(0)
    MethodDict['TotalAssetValue'].append(MethodDict['TotalAssetValue'][-1])

  return MethodDict

def OpenPosition(MethodDict,StockPriceList):
  MethodDict['AssetValue'].append(0)
  MethodDict['AssetStockAmout'].append(10)
  MethodDict['Action'].append(1)
  MethodDict['TotalAssetValue'].append(100)
  return MethodDict

def TradingMethodEvaluate(testDay,AllDailyReturnFrame,StockPriceList,BuyAndHoldDict,GNNDict,ModelData):
  testDayIndex=len(AllDailyReturnFrame)-(TotalTestDay-testDay)+1
  if testDayIndex==len(AllDailyReturnFrame)-(TotalTestDay)+1: #FirstDay
    BuyAndHoldDict=OpenPosition(BuyAndHoldDict,StockPriceList)
    GNNDict=OpenPosition(GNNDict,StockPriceList)
    
    StockPriceList=UpdatePriceList(AllDailyReturnFrame,stockIndex,testDayIndex,Method,StockPriceList)
    GNNDict=Trading(ModelData,stockIndex,StockPriceList[-1],GNNDict)
    
  elif testDayIndex==len(AllDailyReturnFrame)-1:
    StockPriceList=UpdatePriceList(AllDailyReturnFrame,stockIndex,testDayIndex,Method,StockPriceList)
    GNNDict=ClearPosition(GNNDict,StockPriceList)
    BuyAndHoldDict=ClearPosition(BuyAndHoldDict,StockPriceList)
  else:
    StockPriceList=UpdatePriceList(AllDailyReturnFrame,stockIndex,testDayIndex,Method,StockPriceList)

    GNNDict=Trading(ModelData,stockIndex,StockPriceList[-1],GNNDict)
    BuyAndHoldDict=BuyHoldTrading(BuyAndHoldDict,StockPriceList[-1])
  return StockPriceList,BuyAndHoldDict,GNNDict
  

In [ ]:

if __name__ == "__main__":

    import torch
    from torch_geometric.data import Data

    from torch_geometric.datasets import Planetoid
  
    PositiveRelation=[[0,1,0,2],
                 [1,0,2,0]]

    NegativeRelation=[[0,1,0,2],
                 [1,0,2,0]]
    relationMatrix=[PositiveRelation,NegativeRelation]
    TotalTestDay=100
    alloverAccuracy=[]
    dowJohnArray=[]
    predicY=[]
    TrueY=[]
    #trainData,testData=generateData(AllDailyReturnFrame,AllLableFrame,testDay,TotalTestDay,relationMatrix,ClassNum)
    
    Data1List=[]
    Data2List=[]
    targetAssetCorrect=[]
    Method=2
    print("preparing Graph Data.....using Method ",Method)

    InitialAssetValue=100
    InitialAssetAmout=100
    AssetValue=InitialAssetValue;
    AssetAmout=InitialAssetAmout;
    StockPrice=10;
    stockIndex=3
    
    StockPriceList=[10]

    BuyAndHoldDict={
        'AssetValue':[100],
        'AssetStockAmout':[0],
        'Action':[0],
        'TotalAssetValue':[100]
    }

    GNNDict={
        'AssetValue':[100],
        'AssetStockAmout':[0],
         'Action':[0],
        'TotalAssetValue':[100]
    }
    
    print(len(AllDailyReturnFrame))
    
    for testDay in range(TotalTestDay):
      if Method==1:
        alldata=generateDataMethod1(AllDailyReturnFrame,AllLableFrame,testDay,TotalTestDay,relationMatrix,ClassNum)
        
        Data1List.append(alldata)
        Data2List.append(alldata)
        print("Test Day",testDay,"Data",Data1List[testDay],"Test Node Index:" ,Data1List[testDay].test_mask)
      else: 
        traindata,testdata=generateDataMethod2(AllDailyReturnFrame,AllLableFrame,testDay,TotalTestDay,relationMatrix,ClassNum)
        Data1List.append(traindata)
        Data2List.append(testdata)
        # print("Test Day",testDay,"Data",Data1List[testDay],"Predict Y :" ,Data1List[testDay].y)
        print("Test Day",testDay,"Data",Data2List[testDay],"Predict Y:" ,Data2List[testDay].y)
    
    print("Begin to train and evaluate model...")


    ModelList=[]

    train_accuracies_list=[]
    test_accuracies_list=[]
    for testDay in range(TotalTestDay):
      Data1=Data1List[testDay]
      Data2=Data2List[testDay]

      # dataset = Planetoid(root='/tmp/Cora', name='Cora')
      
      #plot_dataset(Data1)
    
      device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
      model = Net(Data1).to(device)
      data1 = Data1.to(device)
      data2=Data2.to(device)

      optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
      
      train_accuracies,test_accuracies=train(data1,data2, plot=False)
      
      train_accuracies_list.append(train_accuracies)
      test_accuracies_list.append(test_accuracies)
      pred,groundtruth=targetAssetTest(data2,stockIndex)
      targetAssetCorrect.append(pred==groundtruth)
      ModelList.append(data2)
      StockPriceList,BuyAndHoldDict,GNNDict=TradingMethodEvaluate(testDay,AllDailyReturnFrame,StockPriceList,BuyAndHoldDict,GNNDict,data2)




    BH_return=(BuyAndHoldDict['AssetValue'][-1]-InitialAssetValue)/InitialAssetValue
    GNN_return=(GNNDict['AssetValue'][-1]-InitialAssetValue)/InitialAssetValue
    print("B&H",BH_return,"GNN",GNN_return)
    
    print("=============")
    print("target Asset Accuracy: ",np.sum(targetAssetCorrect)/len(targetAssetCorrect))
#       6
# [1 0 3 6 3 0]
# [(1, 0), (0, 1), (2, 0), (0, 2), (1, 0), (0, 1), (0, 3), (3, 0), (0, 1), (1, 0)]




In [ ]:
##overfitting?
plt.subplot(2,1,1)
plt.plot(np.mean(train_accuracies_list,axis=0),label='Training Accuracy')
plt.plot(np.mean(test_accuracies_list,axis=0),label='Testing Accuracy')
plt.legend(loc = 'upper right',fontsize=8,ncol=1)
print(np.mean(np.mean(test_accuracies_list,axis=0)))
plt.subplot(2,1,2)
plt.plot(np.mean(test_accuracies_list,axis=0),label='Testing Accuracy')
plt.legend(loc = 'upper right',fontsize=8,ncol=1)
plt.xlabel('Epoch')
plt.savefig('./trainingAccuracy.png')


In [ ]:
if __name__ == "__main__":

    import torch
    from torch_geometric.data import Data

    from torch_geometric.datasets import Planetoid
  
    PositiveRelation=[[0,1,0,2],
                 [1,0,2,0]]

    NegativeRelation=[[0,1,0,2],
                 [1,0,2,0]]
    relationMatrix=[PositiveRelation,NegativeRelation]
    AllSymbolAccuracy=[]
    AllSymbolGroundTruth=[]
    AllSymbolPredict=[]
    for stockIndex in range(len(SymbolList)):
    #for stockIndex in [1]:
        SymbolGroudTruth=[]
        SymbolPrediction=[]
        TotalTestDay=100
        alloverAccuracy=[]
        dowJohnArray=[]
        predicY=[]
        TrueY=[]
        #trainData,testData=generateData(AllDailyReturnFrame,AllLableFrame,testDay,TotalTestDay,relationMatrix,ClassNum)
        
        Method=2
        print("preparing Graph Data.....using Method ",Method)

        InitialAssetValue=100
        InitialAssetAmout=100
        AssetValue=InitialAssetValue;
        AssetAmout=InitialAssetAmout;
        StockPrice=10;
        # stockIndex=12
        targetAssetCorrect=[]
        StockPriceList=[10]
        print("Predict Asset: ",SymbolList[stockIndex])
        BuyAndHoldDict={
            'AssetValue':[100],
            'AssetStockAmout':[0],
            'Action':[0],
            'TotalAssetValue':[100]
        }

        GNNDict={
            'AssetValue':[100],
            'AssetStockAmout':[0],
            'Action':[0],
            'TotalAssetValue':[100]
        }
        
            
        OverallAccuracyTrend=[]

        for testDay in range(TotalTestDay-2):
          
            Data1=Data1List[testDay]
            Data2=Data2List[testDay]

            data2=ModelList[testDay]
            pred,truth=targetAssetTest(data2,stockIndex)
            targetAssetCorrect.append(pred==truth)
            
            SymbolGroudTruth.append(truth.item())
            SymbolPrediction.append(pred.item())
            OverallAccuracyTrend.append(test(data2, train=False))

            StockPriceList,BuyAndHoldDict,GNNDict=TradingMethodEvaluate(testDay,AllDailyReturnFrame,StockPriceList,BuyAndHoldDict,GNNDict,data2)


        AllSymbolGroundTruth.append(SymbolGroudTruth)
        AllSymbolPredict.append(SymbolPrediction)


        BH_return=(BuyAndHoldDict['AssetValue'][-1]-InitialAssetValue)/InitialAssetValue
        GNN_return=(GNNDict['AssetValue'][-1]-InitialAssetValue)/InitialAssetValue
        #print("B&H",BH_return,"GNN",GNN_return)
        
        symbolAccuracy=np.sum(targetAssetCorrect)/len(targetAssetCorrect)
        #print("=============")
        #print("target Asset Accuracy: ",np.sum(targetAssetCorrect)/len(targetAssetCorrect))
        AllSymbolAccuracy.append(symbolAccuracy)

    plt.plot(OverallAccuracyTrend)
    plt.axhline(y=1/ClassNum, xmin=0, xmax=1,color='r')
    plt.axhline(y=np.mean(OverallAccuracyTrend), xmin=0, xmax=1,color='g')
len(AllSymbolGroundTruth[0])
def F_score(AllSymbolGroundTruth,AllSymbolPredict):

  from sklearn.metrics import f1_score
  print(len(AllSymbolGroundTruth))
  F_Micro=[]
  F_Macro=[]
  for isymbol in range(len(SymbolList)):

    f1_micro = f1_score(AllSymbolGroundTruth[isymbol],AllSymbolPredict[isymbol],average='micro')
    f1_macro = f1_score(AllSymbolGroundTruth[isymbol],AllSymbolPredict[isymbol],average='macro')

    F_Micro.append(f1_micro)
    F_Macro.append(f1_macro)
  return F_Micro,F_Macro
#       6
# [1 0 3 6 3 0]
# [(1, 0), (0, 1), (2, 0), (0, 2), (1, 0), (0, 1), (0, 3), (3, 0), (0, 1), (1, 0)]

print(AssociationPara)

print("Ave Acc: ",np.mean(AllSymbolAccuracy),"Corr: ",Corr ,"Assiciation: ",AssociationPara)
print(AllSymbolAccuracy)

F_Micro,F_Macro=F_score(AllSymbolGroundTruth,AllSymbolPredict)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
fig, (ax1, ax2)= plt.subplots(2, 1, sharex='col')


ax1.bar(SymbolList,AllSymbolAccuracy,label="Accuracy",color='#cc6666')

ax1.set_xticks(SymbolList, minor=False)
ax1.set_xticklabels(SymbolList, rotation=90)
ax1.axhline(y=np.mean(AllSymbolAccuracy), xmin=0, xmax=1,color='#FF6666',label='Average accuracy', linestyle='-.')
# ax1.axhline(y=1/ClassNum, xmin=0, xmax=1,color='black',label='baseline')

fig.tight_layout()
plt.subplots_adjust(wspace =0.1, hspace =0.1)#调整子图间距

ax2.bar(SymbolList,F_Micro,label="F Micro")
ax2.axhline(y=np.mean(F_Micro), xmin=0, xmax=1,color='#0099FF',label='Average F Micro', linestyle='-.')
print(np.mean(F_Macro))
ax2.bar(SymbolList,F_Macro,label="F Macro")
ax2.axhline(y=np.mean(F_Macro), xmin=0, xmax=1,color='b',label='Average F Macro', linestyle='-.')
# ax2.axhline(y=1/ClassNum, xmin=0, xmax=1,color='black')
ax2.set_xticks(SymbolList, minor=False)
ax2.set_xticklabels(SymbolList, rotation=90)
lines = []
labels = []

for ax in fig.axes:
    axLine, axLabel = ax.get_legend_handles_labels()
    lines.extend(axLine)
    labels.extend(axLabel)

    
fig.legend(lines, labels,           
           loc = 'upper center',fontsize=8,ncol=3)
fig.savefig('FScore.png')


# data.plot.bar(ax=axes[1,1], color='b', alpha = 0.5)
# data.plot.barh(ax=axes[0,1], color='k', alpha=0.5)






# plt.axhline(y=np.mean(AllSymbolAccuracy), xmin=0, xmax=1,color='g')
# plt.axhline(y=1/ClassNum, xmin=0, xmax=1,color='black')



In [ ]:
fig , ax = plt.subplots()
fig.subplots_adjust(hspace=0.1, wspace=0.1) 
ax2 = ax.twinx()

plt.subplot(4, 1, 1)
plt.plot(StockPriceList)
plt.subplot(4, 1, 2)
plt.plot(GNNDict['Action'])
plt.ylim(ymax=ClassNum-1)
plt.axhline(y=np.floor(ClassNum/2), xmin=0, xmax=1,color='g')

plt.subplot(4, 1, 3)
plt.plot(GNNDict['TotalAssetValue'])
plt.plot(BuyAndHoldDict['TotalAssetValue'])


plt.subplot(4, 1, 4)

plt.plot(GNNDict['AssetStockAmout'])